# Chat with your book

This colab will allow you to ask questions about the story and characters of a book (and maybe as a further step, chat with a chatbot about it)

Enterprise Use Cases:
 - Customer is a Publisher that wants to allow readers to ask questions about the novels in their portfolio in order to determine whether they want to buy them or not
 - Customer is an author that wants to draw attention to their novels by making them queryable
 - Customer is an Agent who wants to learn more about the work of the authors they currently represent or are considering representing
 - Customer wants to provide a service to authors who want to determine if other novels are similar to theirs (finding "comps")

Steps
1) Load and chunk the Book as a PDF from Google Drive

 - Make chunks large and with a lot of overlap so that model has more context

2) Initialize textbison LLM with langchain

3) Generate embeddings from the Book

4) Store the embeddings in a vector store in Matching Engine

5) Use the RetrievalQA chain to send and answer questions about the book by executing the following:

 - 6) Prompt text-bison LLM “Extract the most relevant search terms from [QUESTION]”

 - 7) Send those search terms to the vector store and return the top n results as context

 - 8) Pass that context back to the LLM, giving it a prompt like “please summarize the following results into an answer for [QUESTION]”



In [1]:
# Install Vertex AI LLM SDK# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform==1.25.0

! pip install "shapely<2.0.0"
! pip install protobuf==3.20.3

# Install langchain
! pip install langchain

# PDF loader
! pip install pypdf

# For default embeddings
! pip install tensorflow_hub tensorflow_text

/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found


In [ ]:
import json
import textwrap
# Utils
import time
import uuid
from typing import List

import numpy as np

# Vertex AI
from google.cloud import aiplatform
import vertexai

print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Langchain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel

# Import custom Matching Engine packages
from utils.matching_engine import MatchingEngine
from utils.matching_engine_utils import MatchingEngineUtils

Vertex AI SDK version: 1.25.0
LangChain version: 0.0.259


In [ ]:
PROJECT_ID = "remy-sandbox"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}
STAGING_GCS_BUCKET = "gs://my-bucket/llms" # @param {type:"string"}

In [ ]:
import langchain
print(f"LangChain version: {langchain.__version__}")

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

LangChain version: 0.0.259
Vertex AI SDK version: 1.25.0


In [ ]:
#! pip install pypdf

### Load and chunk the Book

In [ ]:
from langchain.document_loaders import PyPDFLoader
import pypdf

# Load PDF of Book
loaders = [
    PyPDFLoader("./The Council of Light WOTO.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Add document name and source to the metadata
documents = docs

for document in documents:
    doc_md = document.metadata
    document_name = doc_md["source"].split("/")[-1]
    # derive doc source from Document loader
    doc_source_prefix = "/".join(GCS_BUCKET_DOCS.split("/")[:3])
    doc_source_suffix = "/".join(doc_md["source"].split("/")[4:-1])
    source = f"{doc_source_prefix}/{doc_source_suffix}"
    document.metadata = {"source": source, "document_name": document_name}

print(f"# of documents loaded (pre-chunking) = {len(documents)}")

NameError: name 'docs' is not defined

In [ ]:
# split the documents into chunks
# documentation: https://js.langchain.com/docs/modules/indexes/text_splitters/
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
doc_splits = text_splitter.split_documents(docs)

for idx, split in enumerate(doc_splits):
    split.metadata["chunk"] = idx

print(f"# of documents = {len(doc_splits)}")

# of documents = 554


Create functions that will be used later when generating the embeddings from the book


In [ ]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [ ]:
class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

### Initialize Langchain Models

In [ ]:
# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embeddings API integrated with langChain
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

Initialize the Matching Engine Index and Deploy it to an endpoint so that we can send it the embeddings


In [ ]:
ME_REGION = "us-central1"
ME_INDEX_NAME = f"{PROJECT_ID}-me-index"  # @param {type:"string"}
ME_EMBEDDING_DIR = f"{PROJECT_ID}-me-bucket"  # @param {type:"string"}
ME_DIMENSIONS = 768  # when using Vertex PaLM Embedding



In [ ]:
! gsutil mb -l us-central1 gs://$ME_EMBEDDING_DIR

Creating gs://remy-sandbox-me-bucket/...


Create Dummy embeddings in order to initialize the index

In [ ]:
# dummy embedding
init_embedding = {"id": str(uuid.uuid4()), "embedding": list(np.zeros(ME_DIMENSIONS))}

# dump embedding to a local file
with open("embeddings_0.json", "w") as f:
    json.dump(init_embedding, f)

# write embedding to Cloud Storage
! gsutil cp embeddings_0.json gs://remy-sandbox-me-bucket/init_index/embeddings_0.json



Copying file://embeddings_0.json [Content-Type=application/json]...
/ [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


In [ ]:
#Create Index for streaming updates

mengine = MatchingEngineUtils(PROJECT_ID, ME_REGION, ME_INDEX_NAME)

index = mengine.create_index(
    embedding_gcs_uri=f"gs://{ME_EMBEDDING_DIR}/init_index",
    dimensions=ME_DIMENSIONS,
    index_update_method="streaming",
    index_algorithm="tree-ah",
)
if index:
    print(index.name)

INFO:root:Index remy-sandbox-me-index already exists with id projects/811582753906/locations/us-central1/indexes/2861395448403329024


projects/811582753906/locations/us-central1/indexes/2861395448403329024


In [ ]:
#Configure matching engine as vector store

#get ME endpoint ID
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()
print(f"ME_INDEX_ID={ME_INDEX_ID}")
print(f"ME_INDEX_ENDPOINT_ID={ME_INDEX_ENDPOINT_ID}")

ME_INDEX_ID=projects/811582753906/locations/us-central1/indexes/2861395448403329024
ME_INDEX_ENDPOINT_ID=projects/811582753906/locations/us-central1/indexEndpoints/6846518168672796672


In [ ]:
# initialize vector store with the text embeddings model
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=ME_REGION,
    gcs_bucket_name=f'gs://{ME_EMBEDDING_DIR}',
    embedding=embeddings,#calling the text embeddings API
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

In [ ]:
print(ME_EMBEDDING_DIR)

remy-sandbox-me-bucket


In [ ]:
#Deploy Index to Matching Engine

index_endpoint = mengine.deploy_index()
if index_endpoint:
    print(f"Index endpoint resource name: {index_endpoint.name}")
    print(
        f"Index endpoint public domain name: {index_endpoint.public_endpoint_domain_name}"
    )
    print("Deployed indexes on the index endpoint:")
    for d in index_endpoint.deployed_indexes:
        print(f"    {d.id}")

INFO:root:Index endpoint remy-sandbox-me-index-endpoint does not exists. Creating index endpoint...
INFO:root:Deploying index to endpoint with long running operation projects/811582753906/locations/us-central1/indexEndpoints/6846518168672796672/operations/6568716316713156608
INFO:root:Poll the operation to create index endpoint ...
INFO:root:Index endpoint remy-sandbox-me-index-endpoint created with resource name as projects/811582753906/locations/us-central1/indexEndpoints/6846518168672796672 and endpoint domain name as 
INFO:root:Deploying index with request = {'id': 'remy_sandbox_me_index_20230809205125', 'display_name': 'remy_sandbox_me_index_20230809205125', 'index': 'projects/811582753906/locations/us-central1/indexes/2861395448403329024', 'dedicated_resources': {'machine_spec': {'machine_type': 'e2-standard-2'}, 'min_replica_count': 2, 'max_replica_count': 10}}


.

INFO:root:Poll the operation to deploy index ...


...............

INFO:root:Deployed index remy-sandbox-me-index to endpoint remy-sandbox-me-index-endpoint


.Index endpoint resource name: projects/811582753906/locations/us-central1/indexEndpoints/6846518168672796672
Index endpoint public domain name: 
Deployed indexes on the index endpoint:


##generate embeddings and load to store


In [ ]:
doc_splits[0].metadata

{'source': 'file//Council_of_Light', 'document_name': '', 'chunk': 0}

In [ ]:
# Add document name and source to the metadata
for document in documents:
    doc_md = document.metadata
    document_name = doc_md["source"].split("/")[-1]
    # derive doc source from Document loader
    doc_source_prefix = "file"
    doc_source_suffix = "/Council_of_Light"
    source = f"{doc_source_prefix}/{doc_source_suffix}"
    document.metadata = {"source": source, "document_name": document_name}

print(f"# of documents loaded (pre-chunking) = {len(documents)}")
print(document_name)
print(source)

# of documents loaded (pre-chunking) = 314
Council_of_Light
file//Council_of_Light


In [ ]:
#print(doc_splits.metadata["chunk"])


AttributeError: 'list' object has no attribute 'metadata'

In [ ]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in doc_splits]
metadatas = [
    [
        {"namespace": "source", "allow_list": [doc.metadata["source"]]},
        {"namespace": "document_name", "allow_list": "Council of Light"},
        {"namespace": "chunk", "allow_list": [str(doc.metadata["chunk"])]},
    ]
    for doc in doc_splits
]

### Add embeddings to the vector store

In [ ]:

doc_ids = me.add_texts(texts=texts, metadatas=metadatas)

Waiting
.............................................................................................................

INFO:root:Indexed 554 documents to Matching Engine.


In [ ]:
# Test whether search from vector store is working
me.similarity_search("Ben fights Silen", k=2)

Waiting


[Document(page_content='“Hmm\nok,\nwill\nhave\nto\ntweak\nthat\na\nbit.\nHumans\ncan’t\ngo\nvery\nlong\nwithout\nblinking\nbefore\nyour\neyes\nstart\nto\nget\nsuuuuuper\ndry.\nLike\ndeserts,\ncrying\nout\nfor\neven\na\ndrop\nof\nwater .\nCraving\njust\none\nblink…”\nAlfred\ngrowled\nthrough\nhis\nteeth.\nBen\nsmiled\nand\nturned\nback\nto\nLucia.\n“Pretty\ncool\nright?\nImmobilization\nbeams,\nthey\ntarget\nthe\nspine\nand\nrender\nthe\nvictim\ntotally ,\nwell,\nimmobile.\nNot\ncompletely\nimmoble,\nof\ncourse,\nyou\nstill\nneed\nyour\nlungs\nfor\nbreathing\nand\ntongue\nfor\nswallowing\nand\nheart\nfor\nblood\nstuff.\nI\nhave\nto\nadmit,\nit’s\ntaken\na\nfew\niterations\nto\nget\nthat\npart\ncompletely\nright.\nHeh…”\nBen\nran\nhis\nhand\nthrough\nhis\nhair.\n“Anyways,\nI\nhad\nMarim\nset\nup\nthe\ngeneration\ndevices\nwhen\nwe\nfirst\nwalked\nin.”\nBen\nbent\nover\nand\npicked\nup\na\nsmall\ncircular\ndisk\noff\nof\nthe\nfloor .\n“I\nexpected\na\ncouple\nmore\nboonies\nto\nshow\nup\n

In [ ]:
me.similarity_search("What is a com-palm?", k=2, search_distance=0.4)

Waiting


[Document(page_content='attending\nthe\nLabor\nGala,\nthereby\nintroducing\nher\nto\nthe\nExecutive,\nthereby\ncausing\nhim\nto\nmiss\nthe\nmeeting\nthat\nour\nclient\nneeds\nhim\nto\nmiss,\nis\na\ndenominator\nof\n1,000.\nA\nday\nof\nsimulation\nadds\n1\npoint\nto\nour\nnumerator ,\nstill\nkeeping\nus\nwell\nbelow\nthe\nideal\nratio\nof\n1:7.”\n“That’ s\na\npretty\nsimple\ncalculation,\nfor\nyou.”\n“Well\nI\ndumbed\nit\ndown\ntremendously .\nIt’s\nreally\nmore\nof\na\nSunk\nCost\nLinear\nRegression.\nBut\nI\nalso\nhave\na\nSunk\nCost\nRatio\nwhen\nit\ncomes\nto\nexplaining\nthings\nto\nyou,\nand\nthe\ndenominator\nis\npretty\nlow\non\nthat\none.”\n“Does\nmy\nmental\nanguish\nfactor\ninto\neither\nof\nthose\nequations?”\n“Mine\ndoes.”\nShe\nwinked\nat\nme\nand\ntapped\nher\nCom-Palm,\nbeginning\nthe\nsimulation.\nI\nsighed\nand\ntook\nmy\nspot\nin\nthe\ncorner\nof\nthe\nspa\nroom.\nSuddenly ,\nthe\nroom\nfroze\nagain.\n“You\nsee\nwhat\nI\njust\ndid\nthere?”\nArtemis\nasked.\nI\nlooked\

# Retrieve Answers using Retrieval QA chain

In [ ]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.6

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [ ]:
#TODO: Retreival QA portion

In [ ]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions about the plot and characters of a novel.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [ ]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [ ]:
#Enable verbose logging for debugging and troubleshooting the chains which includes the complete prompt to the LLM
# Enable for troubleshooting
qa.combine_documents_chain.verbose = True
qa.combine_documents_chain.llm_chain.verbose = True
qa.combine_documents_chain.llm_chain.llm.verbose = True

In [ ]:
#utility function to format the result
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    result = qa({"query": query})
    return formatter(result)

## Run QA chain on sample questions¶

When you run the query, RetrievalQA chain takes the user question, call the retriever to fetch top k semantically similar texts from the Matching Engine Index (vector store) and passes to the LLM as part of the prompt. The final prompt sent to the LLM looks of this format:

SYSTEM: {system}

=============
{context}
=============

Question: {question}
Helpful Answer:
where:

system: Instructions for LLM on how to respond to the question based on the context
context: Semantically similar text (a.k.a snippets) retreived from the vector store
question: question posed by the user
The response returned from the LLM includes both the response and references that lead to the response. This way the response from LLM is always grounded to the sources. Here we have formatted the response as:

Question: {question}
--------------------------------------------------------------------------------
REFERENCE #n
--------------------------------------------------------------------------------
Matching Score: <score>
Document Source: <document source location>
Document Name: <document file name>
................................................................................
Context:
{}
................................................................................
Response: <answer returned by the LLM>
................................................................................

In [ ]:
ask("Who is the main Character in The Council of Light?")



> Entering new RetrievalQA chain...
Waiting

> Finished chain.
Query: Who is the main Character in The Council of Light?
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.7526170015335083
Document Source: file//Council_of_Light
Document Name: C
................................................................................
Content: 
The Council of Light Book One of the Mechanic and Architect Series By Remy Welch remywelch @gmail.com 135,000 Words
--------------------------------------------------------------------------------
REFERENCE #1
--------------------------------------------------------------------------------
Matching Score: 0.7288496494293213
Document Source: file//Council_of_Light
Document Name: C
.......................................................

In [ ]:
ask("How does the story end?")



> Entering new RetrievalQA chain...
Waiting

> Finished chain.
Query: How does the story end?
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.6619444489479065
Document Source: file//Council_of_Light
Document Name: C
................................................................................
Content: 
"Oh, I don't care," said Lucia, "I've heard worse." "See Dax? You underestimate our dear chef," Ben nudged Lucia's arm.
"But we should get out of here anyway , I don't like the way that guy is looking at me," Ben said, lowering his voice
and gesturing to a man sitting at the table across from them. "Maybe he's imagining Marim sitting on your face," Lucia
said. Ben looked back at her, this time with genuine shock on his face. He burst out suddenly in laughter . 

In [ ]:
ask("What planet is Dax from?")



> Entering new RetrievalQA chain...
Waiting

> Finished chain.
Query: What planet is Dax from?
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.7360135316848755
Document Source: file//Council_of_Light
Document Name: C
................................................................................
Content: 
“An old colleague of mine heard that some of the members of the durious Ruff gang have taken up residence on Taris
proper ,” Dax offered. “The trip will have been well worth it if we snag one of them.” “Now that is a great idea. I bet
Lucia’ s never been to Taris.” Ben yelled out towards the direction Lucia had stormed off, “HEY LUCIA! HAVE YOU BEEN TO
TARIS?” When no response was forthcoming, he opened a line to her Com-Palm. “Hey Lucia. Lucia, Lucia, Luuuuuu

In [ ]:
ask("Who is the villain in the novel?")



> Entering new RetrievalQA chain...
Waiting

> Finished chain.
Query: Who is the villain in the novel?
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.6750061511993408
Document Source: file//Council_of_Light
Document Name: C
................................................................................
Content: 
NOW IF THIS WAS THE SOR T OF STORY where the audience was allowed to ask questions, one might now ask this question:
‘Ben, wasn’ t the point of this whole play to procure the location of the Solar Harnessing Cores so that you can sell
them to the Relliance? Why would you alert Petrus to that threat, even if you’re blaming it on someone else?’ And I
would say, ‘thank you for noticing, how very bright you are, you definitely needed to interrupt me.’ Inde

In [ ]:
#It shouldn't know the answer to this
ask("How does Ben defeat Silen?")




> Entering new RetrievalQA chain...
Waiting

> Finished chain.
Query: How does Ben defeat Silen?
................................................................................
--------------------------------------------------------------------------------
REFERENCE #0
--------------------------------------------------------------------------------
Matching Score: 0.7198085188865662
Document Source: file//Council_of_Light
Document Name: C
................................................................................
Content: 
“It is true, Ben. Remember when that Crucian escaped from the holding cell?” “She was big for a Crucian. And she caught
me by surprise.” “She was four feet tall.” “Ok well now my honor really is insulted. I’m doing this.” He pulled the
bandana around his neck up over his head. It pushed his hair out into a spiky fan around his face, making him look
crazed. “You’re drunk.” “And he’s going to be a little slow from the immobilization beam. See, balance in the u